In [346]:
%%capture
!pip install mediapipe

In [347]:
import numpy as np
import os
import scipy.io as sio
import cv2
import math
from math import cos, sin
import dlib
import glob
from pathlib import Path
import pandas as pd
from PIL import Image, ImageFilter
import random
import mediapipe
import warnings
warnings.filterwarnings('ignore')
from google.colab.patches import cv2_imshow

In [348]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [349]:
!unzip /content/drive/MyDrive/AFLW2000-3D.zip

Archive:  /content/drive/MyDrive/AFLW2000-3D.zip
replace AFLW2000/Code/DrawSolidHead.m? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: AFLW2000/Code/DrawSolidHead.m  
  inflating: AFLW2000/Code/DrawTextureHead.m  
  inflating: AFLW2000/Code/main_show_without_BFM.m  
  inflating: AFLW2000/Code/main_show_with_BFM.m  
  inflating: AFLW2000/Code/Mex/compile.m  
  inflating: AFLW2000/Code/Mex/Tnorm_Vnorm.h  
  inflating: AFLW2000/Code/Mex/Tnorm_VnormC.cpp  
  inflating: AFLW2000/Code/Mex/Tnorm_VnormC.mexw64  
  inflating: AFLW2000/Code/ModelGeneration/ModelGenerate.m  
  inflating: AFLW2000/Code/ModelGeneration/model_info.mat  
  inflating: AFLW2000/Code/Model_Exp.mat  
  inflating: AFLW2000/Code/Model_Shape_Sim.mat  
  inflating: AFLW2000/Code/NormDirection.m  
  inflating: AFLW2000/Code/readme.txt  
  inflating: AFLW2000/Code/RotationMatrix.m  
  inflating: AFLW2000/Code/Tnorm_VnormC.mexw64  
  inflating: AFLW2000/image00002.jpg  
  inflating: AFLW2000/image00002.mat  
  inflating: 

In [350]:
mylist = [Path(f).stem for f in glob.glob("/content/AFLW2000/*.mat")]
mylist

['image02373',
 'image02212',
 'image03832',
 'image03836',
 'image01641',
 'image01643',
 'image01896',
 'image02166',
 'image00670',
 'image02565',
 'image04298',
 'image00298',
 'image02023',
 'image00088',
 'image02739',
 'image03324',
 'image04364',
 'image01820',
 'image03557',
 'image00186',
 'image01676',
 'image03802',
 'image03874',
 'image04027',
 'image00699',
 'image01315',
 'image00310',
 'image03431',
 'image00989',
 'image02768',
 'image04252',
 'image02219',
 'image00222',
 'image02015',
 'image01593',
 'image01050',
 'image01410',
 'image01633',
 'image04163',
 'image00728',
 'image01085',
 'image03043',
 'image03888',
 'image01649',
 'image01126',
 'image01644',
 'image03375',
 'image00547',
 'image00439',
 'image00745',
 'image00788',
 'image03964',
 'image02362',
 'image03663',
 'image01997',
 'image03372',
 'image03780',
 'image01767',
 'image00343',
 'image00454',
 'image04019',
 'image01253',
 'image00057',
 'image00578',
 'image03757',
 'image03726',
 'image005

In [351]:
# choosing random image
#random_file = 'image00053'
data=[]
for random_file in mylist:
  features=[]
  faceModule = mediapipe.solutions.face_mesh
  # loading image and its correspinding mat file
  with faceModule.FaceMesh(static_image_mode=True) as faces:
      # loading the image
      image = cv2.imread('/content/AFLW2000/'+random_file+'.jpg')
      # processing the face to extract the landmark points (468 point) for each x,y,z
      results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      if results.multi_face_landmarks != None: 
        # looping over the faces in the image
        for face in results.multi_face_landmarks:
          for landmark in face.landmark:
            x = landmark.x
            y = landmark.y
            # note: the x and y values are scaled to the their width and height so we will get back their actual value in the image
            shape = image.shape 
            relative_x = int(x * shape[1])
            relative_y = int(y * shape[0])
            # cv2.putText(image, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
            #cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
            features.append(relative_x)
            features.append(relative_y)
  data.append(features)
        #cv2_imshow(image)

In [352]:
len(data)

2000

In [353]:
len(data[17])

936

In [354]:
features = pd.DataFrame(data)

In [355]:
features

,0,1,2,3,4,5,6,7,8,9,...,926,927,928,929,930,931,932,933,934,935
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,235.0,300.0,250.0,279.0,239.0,283.0,242.0,252.0,252.0,272.0,...,248.0,226.0,246.0,229.0,247.0,231.0,275.0,222.0,278.0,219.0
2,251.0,304.0,231.0,288.0,239.0,291.0,206.0,266.0,225.0,280.0,...,221.0,222.0,218.0,227.0,216.0,231.0,256.0,189.0,258.0,184.0
3,236.0,327.0,201.0,306.0,220.0,308.0,180.0,278.0,193.0,298.0,...,205.0,220.0,202.0,227.0,198.0,232.0,242.0,178.0,245.0,169.0
4,203.0,305.0,184.0,277.0,200.0,284.0,185.0,250.0,182.0,269.0,...,225.0,217.0,220.0,221.0,215.0,224.0,262.0,206.0,267.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,207.0,303.0,193.0,277.0,207.0,284.0,197.0,251.0,191.0,269.0,...,237.0,226.0,232.0,229.0,227.0,232.0,273.0,224.0,278.0,220.0
1996,212.0,304.0,202.0,274.0,211.0,282.0,199.0,246.0,200.0,265.0,...,241.0,217.0,235.0,221.0,230.0,224.0,285.0,209.0,290.0,205.0
1997,217.0,292.0,218.0,272.0,219.0,279.0,220.0,241.0,220.0,263.0,...,257.0,225.0,252.0,226.0,249.0,227.0,300.0,227.0,304.0,226.0
1998,238.0,289.0,251.0,266.0,239.0,273.0,241.0,243.0,253.0,259.0,...,240.0,227.0,239.0,229.0,240.0,230.0,257.0,228.0,259.0,225.0


In [356]:
features = features.dropna()

In [357]:
features

,0,1,2,3,4,5,6,7,8,9,...,926,927,928,929,930,931,932,933,934,935
1,235.0,300.0,250.0,279.0,239.0,283.0,242.0,252.0,252.0,272.0,...,248.0,226.0,246.0,229.0,247.0,231.0,275.0,222.0,278.0,219.0
2,251.0,304.0,231.0,288.0,239.0,291.0,206.0,266.0,225.0,280.0,...,221.0,222.0,218.0,227.0,216.0,231.0,256.0,189.0,258.0,184.0
3,236.0,327.0,201.0,306.0,220.0,308.0,180.0,278.0,193.0,298.0,...,205.0,220.0,202.0,227.0,198.0,232.0,242.0,178.0,245.0,169.0
4,203.0,305.0,184.0,277.0,200.0,284.0,185.0,250.0,182.0,269.0,...,225.0,217.0,220.0,221.0,215.0,224.0,262.0,206.0,267.0,200.0
5,226.0,291.0,214.0,277.0,222.0,281.0,212.0,253.0,213.0,271.0,...,244.0,231.0,239.0,233.0,236.0,235.0,280.0,226.0,285.0,223.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,207.0,303.0,193.0,277.0,207.0,284.0,197.0,251.0,191.0,269.0,...,237.0,226.0,232.0,229.0,227.0,232.0,273.0,224.0,278.0,220.0
1996,212.0,304.0,202.0,274.0,211.0,282.0,199.0,246.0,200.0,265.0,...,241.0,217.0,235.0,221.0,230.0,224.0,285.0,209.0,290.0,205.0
1997,217.0,292.0,218.0,272.0,219.0,279.0,220.0,241.0,220.0,263.0,...,257.0,225.0,252.0,226.0,249.0,227.0,300.0,227.0,304.0,226.0
1998,238.0,289.0,251.0,266.0,239.0,273.0,241.0,243.0,253.0,259.0,...,240.0,227.0,239.0,229.0,240.0,230.0,257.0,228.0,259.0,225.0


In [358]:
nose = 45
right_eye = 10
left_eye = 171

In [359]:
for index, row in features.iterrows():
  for i in range(0,935):
      if i%2 ==0:
        row[i]-=row[nose-1]
      else:
        row[i]-=row[nose]
  dist_x = row[right_eye -1] - row[left_eye -1]
  dist_y = row[right_eye] - row[left_eye]
  dist = np.sqrt(dist_x**2 + dist_y**2)
  row /= dist

In [360]:
features

,0,1,2,3,4,5,6,7,8,9,...,926,927,928,929,930,931,932,933,934,935
1,0.107345,0.230477,0.154704,0.164175,0.119974,0.176804,0.129446,0.078930,0.161018,0.142075,...,0.782989,0.713530,0.776675,0.723002,0.779832,0.729317,0.868234,0.700902,0.877706,0.691430
2,0.202033,0.103675,0.148867,0.061142,0.170133,0.069117,0.082408,0.002658,0.132917,0.039875,...,0.587492,0.590150,0.579517,0.603442,0.574200,0.614075,0.680533,0.502425,0.685850,0.489133
3,0.171234,0.137469,0.086823,0.086823,0.132646,0.091646,0.036176,0.019294,0.067529,0.067529,...,0.494407,0.530583,0.487171,0.547465,0.477525,0.559524,0.583641,0.429290,0.590876,0.407584
4,0.032407,0.202542,-0.018904,0.126926,0.024305,0.145830,-0.016203,0.054011,-0.024305,0.105322,...,0.607627,0.586022,0.594124,0.596824,0.580621,0.604926,0.707548,0.556316,0.721050,0.540113
5,0.067891,0.155583,0.033945,0.115980,0.056576,0.127295,0.028288,0.048089,0.031117,0.099007,...,0.690222,0.653448,0.676078,0.659105,0.667592,0.664763,0.792058,0.639304,0.806202,0.630817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000000,0.199880,-0.037311,0.130588,0.000000,0.149244,-0.026651,0.061297,-0.042641,0.109268,...,0.631621,0.602305,0.618295,0.610300,0.604970,0.618295,0.727563,0.596975,0.740888,0.586314
1996,0.058214,0.207907,0.030493,0.124744,0.055442,0.146921,0.022177,0.047126,0.024949,0.099795,...,0.668075,0.601544,0.651442,0.612633,0.637582,0.620949,0.790047,0.579368,0.803907,0.568279
1997,0.041572,0.222709,0.044542,0.163320,0.047511,0.184106,0.050481,0.071267,0.050481,0.136595,...,0.763149,0.668127,0.748302,0.671096,0.739393,0.674065,0.890835,0.674065,0.902713,0.671096
1998,0.115260,0.190023,0.155756,0.118375,0.118375,0.140181,0.124605,0.046727,0.161987,0.096569,...,0.747631,0.707134,0.744516,0.713364,0.747631,0.716480,0.800588,0.710249,0.806818,0.700904


In [361]:
labels =[]
for random_file in mylist:
  label =[]
  image = cv2.imread('/content/AFLW2000/'+random_file+'.jpg')
  # loading the mat file to extract the 3 angels of the face (pitch,yaw,roll)
  mat_file = sio.loadmat('/content/AFLW2000/'+random_file+'.mat')
  # extracting the labels 3 angels
  pose_para = mat_file["Pose_Para"][0][:3]
  pitch = pose_para[0]
  yaw = pose_para[1]
  roll = pose_para[2]
  label.append(pitch)
  label.append(yaw)
  label.append(roll)
  labels.append(label)
  #cv2_imshow(draw_axis(image,pitch,yaw,roll))

In [362]:
len(labels)

2000

In [363]:
len(labels[4])

3

In [364]:
output = pd.DataFrame(labels, columns =['pitch', 'yaw', 'roll'])

In [365]:
output

,pitch,yaw,roll
0,-0.989700,1.358541,-0.944494
1,-0.307462,-0.481678,0.217360
2,-0.123922,0.046326,-0.691047
3,-0.106720,0.435877,-0.715662
4,0.063076,0.709374,-0.220679
...,...,...,...
1995,-0.039597,0.901014,0.011744
1996,-0.077474,0.369024,-0.084786
1997,-0.093383,0.109178,0.244261
1998,-0.044048,-0.738262,0.090391


In [366]:
df = pd.concat([features, output], axis=1, join='inner')

In [367]:
df

,0,1,2,3,4,5,6,7,8,9,...,929,930,931,932,933,934,935,pitch,yaw,roll
1,0.107345,0.230477,0.154704,0.164175,0.119974,0.176804,0.129446,0.078930,0.161018,0.142075,...,0.723002,0.779832,0.729317,0.868234,0.700902,0.877706,0.691430,-0.307462,-0.481678,0.217360
2,0.202033,0.103675,0.148867,0.061142,0.170133,0.069117,0.082408,0.002658,0.132917,0.039875,...,0.603442,0.574200,0.614075,0.680533,0.502425,0.685850,0.489133,-0.123922,0.046326,-0.691047
3,0.171234,0.137469,0.086823,0.086823,0.132646,0.091646,0.036176,0.019294,0.067529,0.067529,...,0.547465,0.477525,0.559524,0.583641,0.429290,0.590876,0.407584,-0.106720,0.435877,-0.715662
4,0.032407,0.202542,-0.018904,0.126926,0.024305,0.145830,-0.016203,0.054011,-0.024305,0.105322,...,0.596824,0.580621,0.604926,0.707548,0.556316,0.721050,0.540113,0.063076,0.709374,-0.220679
5,0.067891,0.155583,0.033945,0.115980,0.056576,0.127295,0.028288,0.048089,0.031117,0.099007,...,0.659105,0.667592,0.664763,0.792058,0.639304,0.806202,0.630817,-0.333410,0.331904,-0.111913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000000,0.199880,-0.037311,0.130588,0.000000,0.149244,-0.026651,0.061297,-0.042641,0.109268,...,0.610300,0.604970,0.618295,0.727563,0.596975,0.740888,0.586314,-0.039597,0.901014,0.011744
1996,0.058214,0.207907,0.030493,0.124744,0.055442,0.146921,0.022177,0.047126,0.024949,0.099795,...,0.612633,0.637582,0.620949,0.790047,0.579368,0.803907,0.568279,-0.077474,0.369024,-0.084786
1997,0.041572,0.222709,0.044542,0.163320,0.047511,0.184106,0.050481,0.071267,0.050481,0.136595,...,0.671096,0.739393,0.674065,0.890835,0.674065,0.902713,0.671096,-0.093383,0.109178,0.244261
1998,0.115260,0.190023,0.155756,0.118375,0.118375,0.140181,0.124605,0.046727,0.161987,0.096569,...,0.713364,0.747631,0.716480,0.800588,0.710249,0.806818,0.700904,-0.044048,-0.738262,0.090391


In [368]:
df.reset_index()

,index,0,1,2,3,4,5,6,7,8,...,929,930,931,932,933,934,935,pitch,yaw,roll
0,1,0.107345,0.230477,0.154704,0.164175,0.119974,0.176804,0.129446,0.078930,0.161018,...,0.723002,0.779832,0.729317,0.868234,0.700902,0.877706,0.691430,-0.307462,-0.481678,0.217360
1,2,0.202033,0.103675,0.148867,0.061142,0.170133,0.069117,0.082408,0.002658,0.132917,...,0.603442,0.574200,0.614075,0.680533,0.502425,0.685850,0.489133,-0.123922,0.046326,-0.691047
2,3,0.171234,0.137469,0.086823,0.086823,0.132646,0.091646,0.036176,0.019294,0.067529,...,0.547465,0.477525,0.559524,0.583641,0.429290,0.590876,0.407584,-0.106720,0.435877,-0.715662
3,4,0.032407,0.202542,-0.018904,0.126926,0.024305,0.145830,-0.016203,0.054011,-0.024305,...,0.596824,0.580621,0.604926,0.707548,0.556316,0.721050,0.540113,0.063076,0.709374,-0.220679
4,5,0.067891,0.155583,0.033945,0.115980,0.056576,0.127295,0.028288,0.048089,0.031117,...,0.659105,0.667592,0.664763,0.792058,0.639304,0.806202,0.630817,-0.333410,0.331904,-0.111913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,1995,0.000000,0.199880,-0.037311,0.130588,0.000000,0.149244,-0.026651,0.061297,-0.042641,...,0.610300,0.604970,0.618295,0.727563,0.596975,0.740888,0.586314,-0.039597,0.901014,0.011744
1849,1996,0.058214,0.207907,0.030493,0.124744,0.055442,0.146921,0.022177,0.047126,0.024949,...,0.612633,0.637582,0.620949,0.790047,0.579368,0.803907,0.568279,-0.077474,0.369024,-0.084786
1850,1997,0.041572,0.222709,0.044542,0.163320,0.047511,0.184106,0.050481,0.071267,0.050481,...,0.671096,0.739393,0.674065,0.890835,0.674065,0.902713,0.671096,-0.093383,0.109178,0.244261
1851,1998,0.115260,0.190023,0.155756,0.118375,0.118375,0.140181,0.124605,0.046727,0.161987,...,0.713364,0.747631,0.716480,0.800588,0.710249,0.806818,0.700904,-0.044048,-0.738262,0.090391


In [369]:
from sklearn.model_selection import train_test_split

In [370]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True) 

In [371]:
x_train = np.array(df_train.drop(columns=['pitch', 'yaw', 'roll']))
y_train_pitch = np.array(df_train['pitch'])
y_train_yaw = np.array(df_train['yaw'])
y_train_roll = np.array(df_train['roll'])

In [372]:
x_test = np.array(df_test.drop(columns=['pitch',	'yaw',	'roll']))
y_test_pitch = np.array(df_test['pitch'])
y_test_yaw = np.array(df_test['yaw'])
y_test_roll = np.array(df_test['roll'])

In [373]:
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, f1_score

In [374]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [375]:
import xgboost
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [376]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

In [377]:
xg_roll = xgboost.XGBRegressor(random_state=0, n_estimators= 520, max_depth= 10, learning_rate= 0.001)

In [378]:
svm = SVR(epsilon=10**(-6))

In [379]:
softvoting_roll = VotingRegressor(estimators=[('xgb', xg_roll),('svm', svm)])

In [380]:
model_roll = softvoting_roll
model_roll.fit(x_train, y_train_roll)
print(model_roll.score(x_train, y_train_roll))
print(model_roll.score(x_test, y_test_roll))
y_test_roll_pred = model_roll.predict(x_test)

[09:55:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.3503392044960555
0.320336636240138


In [381]:
xg_yaw = xgboost.XGBRegressor(random_state=0, n_estimators= 13, max_depth= 10, learning_rate= 0.25)

In [382]:
svm_yaw = SVR(epsilon=0.05)

In [383]:
softvoting_yaw = VotingRegressor(estimators=[('xgb', xg_yaw),('svm', svm_yaw)])

In [384]:
model_yaw = softvoting_yaw 
model_yaw.fit(x_train, y_train_yaw)
print(model_yaw.score(x_train, y_train_yaw))
print(model_yaw.score(x_test, y_test_yaw))
y_test_yaw_pred = model_yaw.predict(x_test)

[09:57:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.9499636305285848
0.9641211650727238


In [385]:
xg_pitch = xgboost.XGBRegressor(random_state=0, n_estimators= 1, max_depth= 37, learning_rate= 0.08)

In [386]:
svm_pitch = SVR(epsilon=0.0001,degree= 6)

In [387]:
rf = RandomForestRegressor(n_estimators = 58, random_state = 0)

In [388]:
dtr = DecisionTreeRegressor(random_state=42)

In [389]:
lasso = Lasso(alpha = 0.5)
lassoCv = LassoCV(cv=3)

In [390]:
softvoting_pitch = VotingRegressor(estimators=[('xgb', xg_pitch),('svm', svm_pitch), ('rf', rf), ('lasso', lasso), ('lassocv', lassoCv)])

In [391]:
model_pitch = softvoting_pitch
# Xg = GridSearchCV(estimator = xg,param_grid = xg_parameters)
model_pitch.fit(x_train, y_train_pitch)
print(model_pitch.score(x_train, y_train_pitch))
print(model_pitch.score(x_test, y_test_pitch))
y_test_pitch_pred = model_pitch.predict(x_test)

[09:57:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
0.2704891087896486
0.14389670923042353


In [392]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img

In [397]:
cap = cv2.VideoCapture('video_.mp4')

if (cap.isOpened()== False): 
  print("Error opening video  file")

#see number of frames
int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

185

In [398]:
# initializing a list to store the frames   
img_array = []
# Read until video is completed
while(cap.isOpened()): 
  # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
      Data=[]
      features=[]
      faceModule = mediapipe.solutions.face_mesh
      # loading image and its correspinding mat file
      with faceModule.FaceMesh(static_image_mode=True) as faces:
        # loading the image
        image = frame
        # processing the face to extract the landmark points (468 point) for each x,y,z
        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None: 
          # looping over the faces in the image
          for face in results.multi_face_landmarks:
            for landmark in face.landmark:
              x = landmark.x
              y = landmark.y
              nose_x = face.landmark[45].x
              nose_y = face.landmark[45].y
              shape = image.shape 
              nose_relative_x = int(nose_x * shape[1])
              nose_relative_y = int(nose_y * shape[0])

              # note: the x and y values are scaled to the their width and height so we will get back their actual value in the image
              shape = image.shape 
              relative_x = int(x * shape[1])
              relative_y = int(y * shape[0])
              # cv2.putText(image, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
              #cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
              features.append(relative_x)
              features.append(relative_y)
      Data.append(features)
              #cv2_imshow(image)

      features = pd.DataFrame(Data)

      if len(Data[0])!=0:
        for i in range(0,935):
          if i%2 ==0:
            features[i] = features[i] - features[nose-1]
          else:
            features[i] = features[i] - features[nose]


        dist_x = features[right_eye -1] - features[left_eye -1]
        dist_y = features[right_eye] - features[left_eye]
        dist = np.sqrt(dist_x**2 + dist_y**2)
        Features=[]
        for x in features:
          x = x / dist
          Features.append(x)

        features = np.array(Features).reshape(1,936)
        image = frame
        # loading the mat file to extract the 3 angels of the face (pitch,yaw,roll)
        # extracting the labels 3 angels

        pitch = model_pitch.predict(features)
        yaw = model_yaw.predict(features)
        roll = model_roll.predict(features)

        image = draw_axis(image,pitch,yaw,roll,nose_relative_x,nose_relative_y)
      else:
        image = frame
      # appending the result frame to the img_array list
      img_array.append(image)
    # Break the loop
    else: 
        break

cap.release()  
# Closes all the frames
cv2.destroyAllWindows()
print("Number of Detected Frames = ",len(img_array))

Number of Detected Frames =  185


In [399]:
#he frames to video
s1, s2, s3 = img_array[0].shape
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('out_video.mp4', fourcc, 25, (s2, s1), True)
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [400]:
from google.colab import files
files.download('out_video.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>